# Análisis inferencial

Este notebook busca crear un análisis inferencial de los datos explorados en la etapa 2. Se busca responde a las siguientes preguntas mediante a la definición de intervalos de confianza y pruebas de hipótesis:

- ¿Qué relación existe entre el nivel educativo y el hábito de lectura de libros en México?
- ¿Cómo influye la disponibilidad de libros en el hogar durante la infancia en los hábitos de lectura en la adultez?

## Definición de valores iniciales

Para el análisis inferencial debemos definir las hipótesis nula y alternativa, así como los valores iniciales que nos permitan realizar las pruebas de hipótesis y calcular los intervalos de confianza.

### Pregunta 1

- **Hipótesis nula (H0)**: Las proporciones de personas que leen son iguales en los distintos niveles educativos.
- **Hipótesis alternativa (H1)**: Al menos una proporción es diferente entre los niveles educativos.

Las proporciones de lectura en los distintos niveles educativos son:
- P_bajo: 0, 1, 2, 3
- P_medio: 4, 5, 6
- P_alto: 7, 8, 9

Las variables que se utilizarán para el análisis son:
- `nivel`: Nivel educativo (1 a 9)
- `p3_1`: Se leyeron libros en los últimos 12 meses
- `p4`: Número de libros leídos en los últimos 12 meses

## Pregunta 2

- **Hipótesis nula (H0)**: Las proporciones de lectura en adultez son iguales entre los niveles de disponibilidad de libros en la infancia.
- **Hipótesis alternativa (H1)**: Al menos una proporción es diferente entre los niveles de disponibilidad de libros en la infancia.

Las proporciones de lectura en los distintos niveles de disponibilidad de libros en la infancia son:
- P_bajo: 0, 1
- P_medio: 2, 3
- P_alto: 4, 5, 6

Las variables que se utilizarán para el análisis son:
- `p34_4`: Disponibilidad de libros en la infancia
- `p34_4_1`: Cantidad de libros de la variante anterior
- `p3_1`: Se leyeron libros en los últimos 12 meses

In [19]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import norm

In [20]:
alpha = 0.05
z = norm.ppf(1 - alpha/2)

def ci_prop(x, n, alpha=0.05):
    z = norm.ppf(1 - alpha/2)
    p = x / n
    se = sqrt(p*(1-p)/n)
    return p, p - z*se, p + z*se

def ci_diff_prop(x1, n1, x2, n2, alpha=0.05):
    z = norm.ppf(1 - alpha/2)
    p1, p2 = x1/n1, x2/n2
    se = sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    d = p1 - p2
    return d, d - z*se, d + z*se

In [21]:
csv_path = "../data/processed/dataframe_limpio.csv"
df = pd.read_csv(csv_path).copy()


df["lector"] = (df["p31"] == 1).astype(int)

def map_edu(n):
    if n <= 4:  return "Básico"
    if n <= 6:  return "Medio"
    return "Superior"

df["edu3"] = df["nivel"].map(map_edu)

def map_inf(x):
    if x in [1, 2]: return "Bajo"
    if x in [3, 4]: return "Medio"
    if x in [5, 6]: return "Alto"
    return np.nan

df["infancia_libros"] = df["p34_4_1"].map(map_inf)


In [22]:
# Proporción total
n = len(df); x = int(df["lector"].sum())
p, lo, hi = ci_prop(x, n, alpha)
print(f"En toda la muestra leen {x} de {n} personas ({p:.1%}).")
print(f"Con 95% de confianza, la proporción real está entre {lo:.1%} y {hi:.1%}.\n")

En toda la muestra leen 319 de 582 personas (54.8%).
Con 95% de confianza, la proporción real está entre 50.8% y 58.9%.



In [23]:
def nota_conclusion(lo, hi):
    # Si el IC completo está arriba de 0 o abajo de 0, la diferencia es clara.
    if lo > 0 or hi < 0:
        return "diferencia clara"
    return "no concluyente"

print("P1. Relación entre nivel educativo y hábito de lectura\n")

# Por grupo educativo
conteos_edu = {}
print("Por nivel educativo:")
for g, sub in df.groupby("edu3"):
    n_g = len(sub); x_g = int(sub["lector"].sum())
    p_g, lo_g, hi_g = ci_prop(x_g, n_g, alpha)
    conteos_edu[g] = (x_g, n_g)
    print(f"- {g}: leen {x_g} de {n_g} ({p_g:.1%}). IC 95%: {lo_g:.1%} a {hi_g:.1%}.")
print()

# Diferencias clave
x_sup, n_sup = conteos_edu.get("Superior", (0, 0))
x_bas, n_bas = conteos_edu.get("Básico",   (0, 0))
x_med, n_med = conteos_edu.get("Medio",    (0, 0))

print("Comparaciones entre grupos:")
if n_sup>0 and n_bas>0:
    d, lo, hi = ci_diff_prop(x_sup, n_sup, x_bas, n_bas, alpha)
    print(f"- Superior vs Básico: diferencia de {d:.1%}. IC 95%: {lo:.1%} a {hi:.1%} "
          f"({nota_conclusion(lo, hi)}).")
else:
    print("- Superior vs Básico: no hay suficientes datos.")

if n_sup>0 and n_med>0:
    d, lo, hi = ci_diff_prop(x_sup, n_sup, x_med, n_med, alpha)
    print(f"- Superior vs Medio: diferencia de {d:.1%}. IC 95%: {lo:.1%} a {hi:.1%} "
          f"({nota_conclusion(lo, hi)}).")
else:
    print("- Superior vs Medio: no hay suficientes datos.")


P1. Relación entre nivel educativo y hábito de lectura

Por nivel educativo:
- Básico: leen 103 de 199 (51.8%). IC 95%: 44.8% a 58.7%.
- Medio: leen 22 de 48 (45.8%). IC 95%: 31.7% a 59.9%.
- Superior: leen 194 de 335 (57.9%). IC 95%: 52.6% a 63.2%.

Comparaciones entre grupos:
- Superior vs Básico: diferencia de 6.2%. IC 95%: -2.6% a 14.9% (no concluyente).
- Superior vs Medio: diferencia de 12.1%. IC 95%: -3.0% a 27.1% (no concluyente).


In [24]:
print("\nP2. Disponibilidad de libros en la infancia y hábito de lectura\n")

grupo_inf = df.dropna(subset=["infancia_libros"]).groupby("infancia_libros")
conteos_inf = {}

for g, sub in grupo_inf:
    n_g = len(sub); x_g = int(sub["lector"].sum())
    p_g, lo_g, hi_g = ci_prop(x_g, n_g, alpha)
    conteos_inf[g] = (x_g, n_g)
    print(f"- {g}: leen {x_g} de {n_g} ({p_g:.1%}). IC 95%: {lo_g:.1%} a {hi_g:.1%}.")

# Diferencia Alto - Bajo
x_alt, n_alt = conteos_inf.get("Alto", (0, 0))
x_baj, n_baj = conteos_inf.get("Bajo", (0, 0))

print("\nComparación clave:")
if n_alt>0 and n_baj>0:
    d, lo, hi = ci_diff_prop(x_alt, n_alt, x_baj, n_baj, alpha)
    print(f"- Alto vs Bajo: diferencia de {d:.1%}. IC 95%: {lo:.1%} a {hi:.1%} "
          f"({nota_conclusion(lo, hi)}).")
else:
    print("- Alto vs Bajo: no hay suficientes datos.")


P2. Disponibilidad de libros en la infancia y hábito de lectura

- Alto: leen 38 de 54 (70.4%). IC 95%: 58.2% a 82.5%.
- Bajo: leen 233 de 437 (53.3%). IC 95%: 48.6% a 58.0%.
- Medio: leen 48 de 91 (52.7%). IC 95%: 42.5% a 63.0%.

Comparación clave:
- Alto vs Bajo: diferencia de 17.1%. IC 95%: 4.0% a 30.1% (diferencia clara).
